In [1]:
!pip install pyvi

In [2]:
!pip install tensorflow
!pip install tensorflow_hub

  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1


In [3]:
import pandas as pd
import re
from pyvi import ViTokenizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [4]:
dataset = pd.read_csv('data.csv')

In [5]:
dataset.head()

,title,category,content
0,Galaxy Gear có thể điều khiển TV thông minh,Công nghệ,Mẫu đồng hồ thông minh Galaxy Gear của Samsung...
1,Nhật – Mỹ định hội đàm chớp nhoáng tại Singapo...,Thế giới,">> Phó Tổng thống Mỹ công du châu Á, bàn về Bi..."
2,Hàng ngàn học sinh miền núi Nghệ An bỏ học,Giáo dục,Hàng ngàn học sinh sau khi tốt nghiệp THCS đã ...
3,'Cô ấy bỏ em vì... em gái trượt đại học?',Đời sống,Mr Búp Bê (BB): Hay là bạn phải đến kèm cho cô...
4,Cậu bé vượt qua Face ID của iPhone ngang anh Q...,Công nghệ,Trang Wired mới đây đưa thông tin cậu bé Ammar...


# Preprocess

In [10]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc) #Pyvi Vitokenizer library
    doc = doc.lower() #Lower
    tokens = doc.split() #Split in_to words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) #Remove all punctuation
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

In [11]:
content = dataset['content'].values
#Create tokenized data
tokens = []
for text in content:
    text = clean_document(text)
    tokens.append(text)

In [12]:
print(tokens[0])

['mẫu', 'đồng_hồ', 'thông_minh', 'galaxy_gear', 'của', 'samsung', 'ảnh', 'afp', 'theo', 'slashgear', 'hôm', '210', 'nếu', 'tương_tác', 'với', 'tv', 'thông_minh', 'thì', 'galaxy_gear', 'có_thể', 'đảm_nhiệm', 'một_số', 'chức_năng', 'như', 'chuyển', 'kênh', 'tăng', 'giảm', 'âm_lượng', 'tắt', 'mở', 'tv', 'ngoài_ra', 'galaxy_gear', 'còn', 'có', 'khả_năng', 'theo_dõi', 'sức_khỏe', 'tức_là', 'các', 'thông_tin', 'về', 'sức_khỏe', 'của', 'người', 'dùng', 'khi', 'đeo', 'thiết_bị', 'sẽ', 'được', 'phát', 'đến', 'tv', 'để', 'họ', 'nắm', 'được', 'tình_trạng', 'sức_khỏe', 'của', 'mình', 'được', 'biết', 'hiện', 'galaxy_gear', 'chỉ', 'có_thể', 'làm_việc', 'trên', 'các', 'thiết_bị', 'di_động', 'của', 'samsung', 'như', 'galaxy_note', '3', 'và', 'galaxy_tab', '101', 'inch', 'thời_gian', 'tới', 'các', 'thiết_bị', 'khác', 'như', 'galaxy_s4', 'galaxy_note', '2', 'cũng', 'sẽ', 'có', 'bản', 'cập_nhật', 'để', 'tương_tác', 'được', 'với', 'galaxy_gear', 'thành', 'luân']


# Doc2Vec

In [13]:
def process_doc2vec_similarity(tokens, document):
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokens)]
    model = Doc2Vec(documents, vector_size=20, window=2, min_count=1, workers=4)
    
    doc_token = clean_document(document)
    doc_token = list(filter(lambda x: x in model.wv.key_to_index.keys(), doc_token))
    doc_vector = model.infer_vector(doc_token)
    
    vectors = []
    
    for token in tokens:
        vectors.append(model.infer_vector(token))
    scores = cosine_similarity([doc_vector], vectors).flatten()
    
    highest_score = 0
    highest_score_index = 0
    for i, score in enumerate(scores):
        if highest_score < score:
            highest_score = score
            highest_score_index = i

    most_similar_document = dataset['content'].values[highest_score_index]

    print("Most similar document by Doc2Vec with the score:", most_similar_document, highest_score)

In [14]:
process_doc2vec_similarity(tokens, "Hai model mới, Galaxy A34 5G và A54 5G được Samsung giới thiệu cùng thời điểm ra mắt toàn cầu. Việt Nam thuộc nhóm thị trường được ưu tiên mở bán dòng sản phẩm này sớm. Điều này là cần thiết bởi Samsung đã nới rộng vòng đời sản phẩm Galaxy A từ 6 tháng thành một năm. Đồng thời, công ty cũng nâng giá của A34 và A54 lên cao hơn khoảng 500.000-1 triệu đồng so với thế hệ trước. Mẫu Galaxy A34 5G khởi điểm từ mốc 8,5 triệu đồng, A54 5G là 10,5 triệu đồng. Đối thủ chính của 2 sản phẩm này là dòng Xiaomi 13 Lite cũng mới ra mắt. Thiết bị này sở hữu thiết kế bắt mắt và đèn flash để selfie, hướng đến đối tượng khách hàng nữ giới. Vivo V25e hay Reno 8T cũng là những lựa chọn sáng giá khác ở phân khúc này. Mặt khác, không chỉ có các hãng Android cạnh tranh ở phân khúc trung cấp. iPhone 11 chính hãng sau nhiều đợt điều chỉnh cũng về chung tầm giá này. Người dùng hiện nhiều lựa chọn hơn khi mua máy. Galaxy A34 và A54 mới ra mắt có thay đổi lớn về ngoại hình. Thiết bị có khung viền vuông vức và cụm camera tách rời, lấy cảm hứng từ Galaxy S. Sản phẩm chuyển sang sử dụng nền tảng Exynos và MediaTek mới thay vì Snapdragon. Ngoài ra, Samsung cũng hứa sẽ cập nhật phần mềm Android đến 4 năm và 5 năm vá bảo mật cho hai dòng máy tầm trung. Galaxy A34 và A54 là khởi đầu của dòng Galaxy A mới trong năm 2023. Theo nhiều tin đồn, nhà sản xuất Hàn Quốc sẽ sớm lấp đầy dải sản phẩm bằng các mẫu A24 hay A84 trong thời gian tới.")

Most similar document by Doc2Vec with the score: LG V30 sẽ được bán ra tại Hàn Quốc trong tuần này.
LG V30 là mẫu smartphone flagship mới ra mắt hồi tháng 8/2017 của ông lớn Hàn Quốc và được LG kỳ vọng giúp tạo cú hích cho mảng kinh doanh điện thoại vốn đang tụt dốc thảm hại của hãng.
Được lột xác nhờ thiết kế mới với viền siêu mỏng và màn hình OLED FullVision được hãng gọi là P-OLED, tỷ lệ 18:9. Kích thước lên đến 6-inch với độ phân giải 1440 x 2880 pixels, mật độ điểm ảnh 537ppi, LG-V30 hứa hẹn sẽ tạo ra sự cạnh tranh khá uy lực với Samsung Galaxy S8, Note 8 hoặc iPhone 8, iPhone 8 Plus trên thị trường.
Được trang bị cặp camera sau "khủng" hứa hẹn sẽ làm hài lòng người dùng.
Với việc được hãng định giá 840 USD trên thị trường quê nhà Hàn Quốc, LG đang cho mình một lợi thế đáng kể trước đối thủ Galaxy Note 8 đến từ Samsung (giá trên 1000 USD). Nhiều chuyên gia nhận định chắc chắn, LG V30 sẽ có được doanh số "khủng" khi bán ra thị trường cuối tuần này.
Sau khi chính thức bán ra tại thị

# Jaccard score

In [15]:
def calculate_jaccard(word_tokens1, word_tokens2):
    # Combine both tokens to find union.
    both_tokens = word_tokens1 + word_tokens2
    union = set(both_tokens)

    # Calculate intersection.
    intersection = set()
    for w in word_tokens1:
        if w in word_tokens2:
            intersection.add(w)

    jaccard_score = len(intersection)/len(union)
    return jaccard_score

In [16]:
def process_jaccard_similarity(base_tokens, document):
    doc_token = clean_document(document)
    scores = []
    for token in base_tokens:
        score = calculate_jaccard(token, doc_token)
        scores.append(score)

    highest_score = 0
    highest_score_index = 0
    for i, score in enumerate(scores):
        if highest_score < score:
            highest_score = score
            highest_score_index = i

    most_similar_document = dataset['content'].values[highest_score_index]

    print("Most similar document by Jaccard with the score:", most_similar_document, highest_score)

In [17]:
process_jaccard_similarity(tokens, "Hai model mới, Galaxy A34 5G và A54 5G được Samsung giới thiệu cùng thời điểm ra mắt toàn cầu. Việt Nam thuộc nhóm thị trường được ưu tiên mở bán dòng sản phẩm này sớm. Điều này là cần thiết bởi Samsung đã nới rộng vòng đời sản phẩm Galaxy A từ 6 tháng thành một năm. Đồng thời, công ty cũng nâng giá của A34 và A54 lên cao hơn khoảng 500.000-1 triệu đồng so với thế hệ trước. Mẫu Galaxy A34 5G khởi điểm từ mốc 8,5 triệu đồng, A54 5G là 10,5 triệu đồng. Đối thủ chính của 2 sản phẩm này là dòng Xiaomi 13 Lite cũng mới ra mắt. Thiết bị này sở hữu thiết kế bắt mắt và đèn flash để selfie, hướng đến đối tượng khách hàng nữ giới. Vivo V25e hay Reno 8T cũng là những lựa chọn sáng giá khác ở phân khúc này. Mặt khác, không chỉ có các hãng Android cạnh tranh ở phân khúc trung cấp. iPhone 11 chính hãng sau nhiều đợt điều chỉnh cũng về chung tầm giá này. Người dùng hiện nhiều lựa chọn hơn khi mua máy. Galaxy A34 và A54 mới ra mắt có thay đổi lớn về ngoại hình. Thiết bị có khung viền vuông vức và cụm camera tách rời, lấy cảm hứng từ Galaxy S. Sản phẩm chuyển sang sử dụng nền tảng Exynos và MediaTek mới thay vì Snapdragon. Ngoài ra, Samsung cũng hứa sẽ cập nhật phần mềm Android đến 4 năm và 5 năm vá bảo mật cho hai dòng máy tầm trung. Galaxy A34 và A54 là khởi đầu của dòng Galaxy A mới trong năm 2023. Theo nhiều tin đồn, nhà sản xuất Hàn Quốc sẽ sớm lấp đầy dải sản phẩm bằng các mẫu A24 hay A84 trong thời gian tới.")

Most similar document by Jaccard with the score: Điều khó tin đang xuất hiện tại thị trường Việt Nam khi giá iPhone 6 Plus chính hãng có xu hướng rẻ hơn so với iPhone 6. Một nhà bán lẻ trong nước đang cho bán iPhone 6 Plus chính hãng giá 11,99 triệu đồng. Tuy nhiên, nếu mua online, giá bán của máy giảm xuống thành 11,49 triệu đồng.
Trong khi đó, mẫu iPhone 6 hiện có giá bán 11,99 triệu đồng. Tính ra, người dùng có thể tiết kiệm 500.000 đồng nếu mua iPhone 6 Plus so với iPhone 6.
Giá bán iPhone 6 Plus đang thấp hơn cả iPhone 6. Ảnh chụp màn hình.
Đây là chuyện khá hy hữu trên thị trường bởi từ thời điểm cuối 2014 (khi máy lên kệ), giá bán của iPhone 6 Plus luôn cao hơn khoảng 2,5 triệu đồng so với iPhone 6.
Theo thông tin từ các đại lý trong nước, iPhone 6 16 GB có dấu hiệu khan hàng từ cuối năm ngoái. Một số nơi hiện không có sản phẩm để bán.
Bản thân đại lý lớn nhất Việt Nam cũng chỉ mới nhập được một đợt hàng cách đây vài hôm. Trong khi đó, nguồn hàng iPhone 6 Plus 16 GB vẫn ổn định.

# TF-IDF 

In [18]:
def process_tfidf_similarity(document):
    vectorizer = TfidfVectorizer()
    base_docs = list(dataset['content'].values)

    # To make uniformed vectors, both documents need to be combined first.
    base_docs.insert(0, document)
    embeddings = vectorizer.fit_transform(base_docs)

    scores = cosine_similarity(embeddings[0:1], embeddings[1:]).flatten()

    highest_score = 0
    highest_score_index = 0
    for i, score in enumerate(scores):
        if highest_score < score:
            highest_score = score
            highest_score_index = i


    most_similar_document = dataset['content'].values[highest_score_index]

    print("Most similar document by TF-IDF with the score:", most_similar_document, highest_score)

In [19]:
process_tfidf_similarity("Hai model mới, Galaxy A34 5G và A54 5G được Samsung giới thiệu cùng thời điểm ra mắt toàn cầu. Việt Nam thuộc nhóm thị trường được ưu tiên mở bán dòng sản phẩm này sớm. Điều này là cần thiết bởi Samsung đã nới rộng vòng đời sản phẩm Galaxy A từ 6 tháng thành một năm. Đồng thời, công ty cũng nâng giá của A34 và A54 lên cao hơn khoảng 500.000-1 triệu đồng so với thế hệ trước. Mẫu Galaxy A34 5G khởi điểm từ mốc 8,5 triệu đồng, A54 5G là 10,5 triệu đồng. Đối thủ chính của 2 sản phẩm này là dòng Xiaomi 13 Lite cũng mới ra mắt. Thiết bị này sở hữu thiết kế bắt mắt và đèn flash để selfie, hướng đến đối tượng khách hàng nữ giới. Vivo V25e hay Reno 8T cũng là những lựa chọn sáng giá khác ở phân khúc này. Mặt khác, không chỉ có các hãng Android cạnh tranh ở phân khúc trung cấp. iPhone 11 chính hãng sau nhiều đợt điều chỉnh cũng về chung tầm giá này. Người dùng hiện nhiều lựa chọn hơn khi mua máy. Galaxy A34 và A54 mới ra mắt có thay đổi lớn về ngoại hình. Thiết bị có khung viền vuông vức và cụm camera tách rời, lấy cảm hứng từ Galaxy S. Sản phẩm chuyển sang sử dụng nền tảng Exynos và MediaTek mới thay vì Snapdragon. Ngoài ra, Samsung cũng hứa sẽ cập nhật phần mềm Android đến 4 năm và 5 năm vá bảo mật cho hai dòng máy tầm trung. Galaxy A34 và A54 là khởi đầu của dòng Galaxy A mới trong năm 2023. Theo nhiều tin đồn, nhà sản xuất Hàn Quốc sẽ sớm lấp đầy dải sản phẩm bằng các mẫu A24 hay A84 trong thời gian tới.")

Most similar document by TF-IDF with the score: Công ty Điện tử Samsung Vina vừa công bố số liệu thống kê lượng máy bán ra của các sản phẩm thuộc dòng Galaxy A 2016, cụ thể là Galaxy A3, Galaxy A5 và Galaxy A7 ra mắt trong nửa đầu năm nay. Theo đó, sự yêu thích mà người dùng dành cho dòng Galaxy A 2016 được thể hiện qua doanh số lớn mà 3 sản phẩm trên mang lại so với các thiết bị tiền nhiệm năm 2015.
Galaxy A5 2016 đi đầu doanh thu trong dòng Galaxy A.
Galaxy A5 2016 gây ấn tượng mạnh khi doanh số tăng đến 74% so với Galaxy A5 2015. Nối tiếp là Galaxy A3 2016 với doanh số vượt xa Galaxy A3 2015 43%. Về cuối cùng nhưng ấn tượng không kém là Galaxy A7 2016, với doanh số tăng 37% so với Galaxy A7 2015. Tỷ lệ tăng trưởng này cũng phản ánh sự yêu thích của người dùng đối với từng sản phẩm trong dòng Galaxy A 2016, theo thứ tự lần lượt là Galaxy A5, Galaxy A3 và Galaxy A7.
Phiên bản máy màu vàng (gold) được ưa chuộng nhất.
Tỷ lệ các phiên bản màu của dòng Galaxy A 2016 cũng được thể hiện rõ 

# USE (Tensorflow)

In [20]:
import tensorflow as tf
import tensorflow_hub as hub


def process_use_similarity(document):
    filename = "https://tfhub.dev/google/universal-sentence-encoder/4"

    model = hub.load(filename)

    doc_embeddings = model([document])

    embeddings = model(tokens)

    scores = cosine_similarity(doc_embeddings, embeddings).flatten()

    highest_score = 0
    highest_score_index = 0
    for i, score in enumerate(scores):
        if highest_score < score:
            highest_score = score
            highest_score_index = i

    most_similar_document = dataset['content'].values[highest_score_index]
    print("Most similar document by USE with the score:", most_similar_document, highest_score)

In [ ]:
process_use_similarity("Hai model mới, Galaxy A34 5G và A54 5G được Samsung giới thiệu cùng thời điểm ra mắt toàn cầu. Việt Nam thuộc nhóm thị trường được ưu tiên mở bán dòng sản phẩm này sớm. Điều này là cần thiết bởi Samsung đã nới rộng vòng đời sản phẩm Galaxy A từ 6 tháng thành một năm. Đồng thời, công ty cũng nâng giá của A34 và A54 lên cao hơn khoảng 500.000-1 triệu đồng so với thế hệ trước. Mẫu Galaxy A34 5G khởi điểm từ mốc 8,5 triệu đồng, A54 5G là 10,5 triệu đồng. Đối thủ chính của 2 sản phẩm này là dòng Xiaomi 13 Lite cũng mới ra mắt. Thiết bị này sở hữu thiết kế bắt mắt và đèn flash để selfie, hướng đến đối tượng khách hàng nữ giới. Vivo V25e hay Reno 8T cũng là những lựa chọn sáng giá khác ở phân khúc này. Mặt khác, không chỉ có các hãng Android cạnh tranh ở phân khúc trung cấp. iPhone 11 chính hãng sau nhiều đợt điều chỉnh cũng về chung tầm giá này. Người dùng hiện nhiều lựa chọn hơn khi mua máy. Galaxy A34 và A54 mới ra mắt có thay đổi lớn về ngoại hình. Thiết bị có khung viền vuông vức và cụm camera tách rời, lấy cảm hứng từ Galaxy S. Sản phẩm chuyển sang sử dụng nền tảng Exynos và MediaTek mới thay vì Snapdragon. Ngoài ra, Samsung cũng hứa sẽ cập nhật phần mềm Android đến 4 năm và 5 năm vá bảo mật cho hai dòng máy tầm trung. Galaxy A34 và A54 là khởi đầu của dòng Galaxy A mới trong năm 2023. Theo nhiều tin đồn, nhà sản xuất Hàn Quốc sẽ sớm lấp đầy dải sản phẩm bằng các mẫu A24 hay A84 trong thời gian tới.")

# BERT (Google's Transformer)

In [23]:
!pip install sentence_transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=8ac4c382b08a435a2f5ae1712774b60a486cd3306a749e33fa81f13866d904fd
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
import nltk
nltk.download('punkt')

In [25]:
import numpy as np
from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer